In [ ]:
# /---------------------------------------------------------------
# Course            : Big Data Analytics
# Course Code       : CDB3034
# Assignment        : 2
# Group             : 1
# Student Name 1    : Chan Seow Fen / 0207368
# Student Name 2    : Cheah Pin Chee / 0197637
# Student Name 3    : Ong Yi Wen / 0207333
# Student Name 4    : Saw Keat Loon / 0207778
# /---------------------------------------------------------------
# Data source: https://openlearning.uowmkdu.edu.my/courses/pg-cbd-3034n-big-data-analysis-jjoshua/data_a2/?cl=1
# Original Data Files: T_Data_C1.csv, T_Data_C2.csv, T_Data_C3.csv
# /---------------------------------------------------------------
from pyspark.sql.functions import col, when, count, avg
from pyspark.ml.feature import Imputer, StringIndexer, VectorAssembler, OneHotEncoder
import matplotlib.pyplot as plt
import seaborn as sns

default_file_store_path = "dbfs:/FileStore/"

# Task 2: Customer Retention Prediction and Display Results
# Load the cleaned data
df_final = spark.read.csv("dbfs:/FileStore/tables/BDA_T_data_Final.csv", header=True, inferSchema=True)
# Drop same customer ID
df_final = df_final.dropDuplicates(subset=["CustomerID"])
print("Data size of BDA_T_data_Merged.csv: ", df_final.count(), " rows", " and ", len(df_final.columns), " columns")
display(df_final)

In [ ]:
# Task 2.1 Find the number of churn users are in the dataset, discuss your opinion on the output whether the given dataset is balanced or imbalanced. What will be the descriptive statistics for tenure, totalcharges, and monthly charges?

# Find the total user, churn user, and no churn user in the dataset
total_user = df_final.count()
churn_count = df_final.filter(col("Churn") == "Yes").count()
no_churn_count = df_final.filter(col("Churn") == "No").count()
print("Total number of users in the dataset     : ", total_user)
print("Number of churn users in the dataset     : ", churn_count)
print("Number of no churn users in the dataset  : ", no_churn_count)

# Calculate ratio
churn_ratio = churn_count / total_user * 100
no_churn_ratio = no_churn_count / total_user * 100


# Plot pie chart with value labeled to visualize the churn ratio with matplotlib
labels = 'Churn', 'No Churn'
sizes = [churn_count, no_churn_count]
colors = ['royalblue', 'lightcoral']
explode = (0.1, 0)  # explode 1st slice
plt.pie(sizes, explode=explode, labels=labels, colors=colors, autopct=lambda pct: f"{pct:.1f}%\n({int(pct/100 * sum(sizes)):.0f})", startangle=140)
plt.axis('equal')
plt.title("Churn Ratio")
plt.show()

# Descriptive statistics for tenure, totalcharges, and monthly charges
df_final.describe(["tenure", "TotalCharges", "MonthlyCharges"]).display()

# Insight with histogram for each feature and visualize the distribution of tenure, totalcharges, and monthly charges
# For Tenure
tenure = df_final.select("tenure").rdd.flatMap(lambda x: x).collect()
counts, bins, patches = plt.hist(tenure, bins=30, color='royalblue', edgecolor='white', linewidth=0.2)
plt.title("Tenure Distribution")
plt.xlabel("Tenure (Months)")
plt.ylabel("Customer Count")
# Display the count for each bin
for count, bin in zip(counts, bins):
    plt.text(bin, count, str(int(count)), fontsize=8, color='black', weight='bold')
plt.show()
# For TotalCharges
totalcharges = df_final.select("TotalCharges").rdd.flatMap(lambda x: x).collect()
counts, bins, patches = plt.hist(totalcharges, bins=30, color='royalblue', edgecolor='white', linewidth=0.2)
plt.title("TotalCharges Distribution")
plt.xlabel("TotalCharges ($)")
plt.ylabel("Customer Count")
# Display the count for each bin
for count, bin in zip(counts, bins):
    plt.text(bin, count, str(int(count)), fontsize=8, color='black', weight='bold')
plt.show()
# For MonthlyCharges
monthlycharges = df_final.select("MonthlyCharges").rdd.flatMap(lambda x: x).collect()
counts, bins, patches = plt.hist(monthlycharges, bins=30, color='royalblue', edgecolor='white', linewidth=0.2)
plt.title("MonthlyCharges Distribution")
plt.xlabel("MonthlyCharges ($)")
plt.ylabel("Customer Count")
# Display the count for each bin
for count, bin in zip(counts, bins):
    plt.text(bin, count, str(int(count)), fontsize=8, color='black', weight='bold')
plt.show()

In [ ]:
# Task 2.2 Do the analysis on a) Male, Female, churn, and b) SeniorCitizen, churn out of the two which is the most suitable attribute that you can keep for further analysis? Does the tenure correlate to people become churn? Pivot the values and plot the values for better representation.
# 1.1 Plot data for Churn by Gender
gender_churn = df_final.groupBy("gender", "Churn").count().orderBy("gender", "Churn")
gender_churn = gender_churn.toPandas()
sns.set_style("whitegrid")
ax = sns.barplot(x="gender", y="count", hue="Churn", data=gender_churn)
plt.title("Churn Analysis By Gender")
plt.xlabel("Gender")
plt.ylabel("Number of Customer")
for p in ax.patches:
    ax.annotate(f'{int(p.get_height())}', (p.get_x() + p.get_width() / 2.0, p.get_height()), ha='center', va='baseline')
plt.show()

# 1.2 Churned Analysis for each gender by tenure, Monthly Charges and Total Charges
churned = df_final.filter(df_final.Churn == 'Yes').select('gender', 'tenure', 'TotalCharges', 'MonthlyCharges')
male_churned = churned.filter(churned.gender == 'Male')
female_churned = churned.filter(churned.gender == 'Female')

# Plot the tenure, Total Charges and Monthly Charges for Male
plt.scatter(male_churned.select('tenure').rdd.flatMap(lambda x: x).collect(), male_churned.select('MonthlyCharges').rdd.flatMap(lambda x: x).collect(), color='royalblue')
plt.xlabel('Tenure (Months)')
plt.ylabel('Monthly Charges ($)')
plt.title('Male Churned Customers by Tenure and Monthly Charges')
plt.show()

plt.scatter(male_churned.select('tenure').rdd.flatMap(lambda x: x).collect(), male_churned.select('TotalCharges').rdd.flatMap(lambda x: x).collect(), color='royalblue')
plt.xlabel('Tenure (Months)')
plt.ylabel('Total Charges ($)')
plt.title('Male Churned Customers by Tenure and Total Charges')
plt.show()

# Plot the tenure, Total Charges and Monthly Charges for Female
plt.scatter(female_churned.select('tenure').rdd.flatMap(lambda x: x).collect(), female_churned.select('MonthlyCharges').rdd.flatMap(lambda x: x).collect(), color='lightcoral')
plt.xlabel('Tenure (Months)')
plt.ylabel('Monthly Charges ($)')
plt.title('Female Churned Customers by Tenure and Monthly Charges')
plt.show()

plt.scatter(female_churned.select('tenure').rdd.flatMap(lambda x: x).collect(), female_churned.select('TotalCharges').rdd.flatMap(lambda x: x).collect(), color='lightcoral')
plt.xlabel('Tenure (Months)')
plt.ylabel('Total Charges ($)')
plt.title('Female Churned Customers by Tenure and Total Charges')
plt.show()

# 2.1 Plot data for Churn by SeniorCitizen
senior_churn = df_final.groupBy("SeniorCitizen", "Churn").count().orderBy("SeniorCitizen", "Churn")
senior_churn = senior_churn.toPandas()
senior_churn['SeniorCitizen'] = senior_churn['SeniorCitizen'].replace({0: 'No', 1: 'Yes'})
sns.set_style("whitegrid")
ax = sns.barplot(x="SeniorCitizen", y="count", hue="Churn", data=senior_churn)
plt.title("Churn Analysis By Senior Citizen")
plt.xlabel("Senior Citizen")
plt.ylabel("Number of Customer")
for p in ax.patches:
    ax.annotate(f'{int(p.get_height())}', (p.get_x() + p.get_width() / 2.0, p.get_height()), ha='center', va='baseline')
plt.show()

# 2.2 Churned Analysis for each Senior Citizen by tenure, Monthly Charges and Total Charges
senior_churned = df_final.filter((df_final.Churn == 'Yes') & (df_final.SeniorCitizen == 1)).select('SeniorCitizen', 'tenure', 'TotalCharges', 'MonthlyCharges')

plt.scatter(senior_churned.select('tenure').rdd.flatMap(lambda x: x).collect(), senior_churned.select('MonthlyCharges').rdd.flatMap(lambda x: x).collect(), color='blue')
plt.xlabel('Tenure (Months)')
plt.ylabel('Monthly Charges ($)')
plt.title('Senior Churned Customers by Tenure and Monthly Charges')
plt.show()

plt.scatter(senior_churned.select('tenure').rdd.flatMap(lambda x: x).collect(), senior_churned.select('TotalCharges').rdd.flatMap(lambda x: x).collect(), color='pink')
plt.xlabel('Tenure (Months)')
plt.ylabel('Total Charges ($)')
plt.title('Senior Churned Customers by Tenure and Total Charges')
plt.show()

# 3.1 Calculate the correlation between tenure and churn
# Convert the Churn column to numeric value, replace Yes with 1 and No with 0
df_pivot = df_final.withColumn("Churn", col("Churn").cast("string"))
df_pivot = df_pivot.withColumn("Churn", when(col("Churn") == "Yes", 1).otherwise(0))
correlation = df_pivot.stat.corr("tenure", "Churn")
print("Correlation between tenure and churn: ", correlation)

# Pivot the values and plot the values for better representation
pivot = df_final.groupBy("tenure").pivot("Churn").count().orderBy("tenure")
pivot = pivot.toPandas()
pivot = pivot.fillna(0)
pivot = pivot.set_index('tenure')
pivot.plot(kind='bar', stacked=True, figsize=(20, 10))
plt.title("Churn Analysis by Tenure")
plt.xlabel("Tenure (Months)")
plt.ylabel("Number of Customer")
plt.show()

# 4.1 Distribution of tenure by Churn
churned = df_final.filter(col('Churn') == 'Yes').agg(count('Churn')).collect()[0][0]
not_churned = df_final.filter(col('Churn') == 'No').agg(count('Churn')).collect()[0][0]
# Compute the probability density estimate (KDE) of the distribution of tenure for each churn group
churned_kde = sns.kdeplot(df_final.filter(col('Churn') == 'Yes').select('tenure').toPandas()['tenure'], shade=True, label='Churned')
not_churned_kde = sns.kdeplot(df_final.filter(col('Churn') == 'No').select('tenure').toPandas()['tenure'], shade=True, label='Not churned')
# Calculate the average tenure for each churn group
churned_avg_tenure = df_final.filter(col('Churn') == 'Yes').agg(avg('tenure')).collect()[0][0]
not_churned_avg_tenure = df_final.filter(col('Churn') == 'No').agg(avg('tenure')).collect()[0][0]
# Plot the average tenure for each churn group
plt.axvline(churned_avg_tenure, color='lightcoral', linestyle='--', label='Avg tenure (Churned)')
plt.axvline(not_churned_avg_tenure, color='lime', linestyle='--', label='Avg tenure (Not churned)')
plt.title('Distribution of Tenure by Churn')
plt.xlabel('Tenure (Months)')
plt.ylabel('Probability density')
plt.legend()
plt.show()

In [ ]:
# Task 2.3	Use a model building process with Databricks for the given data, create a suitable training, evaluation data items, then build a pipeline with suitable transformers (onehotencoder, stringindexer, vectorassembler)
spark_dr = spark_dr.drop("customerID")

spark_dr = spark_dr.withColumn("tenure", col("tenure").cast("double")).withColumn("MonthlyCharges", col("MonthlyCharges").cast("double")).withColumn("TotalCharges", col("TotalCharges").cast("double"))

imputer = Imputer(inputCols=["TotalCharges"], outputCols=["TotalCharges"], strategy="mean")  # Or "median"

spark_dr = imputer.fit(spark_dr).transform(spark_dr)
trainDF, testDF = spark_dr.randomSplit([0.8, 0.2], seed=42)
print(trainDF.cache().count())  # Cache because accessing training data multiple times
print(testDF.count())

categoricalCols = [
    "gender",
    "SeniorCitizen",
    "Partner",
    "Dependents",
    "PhoneService",
    "MultipleLines",
    "InternetService",
    "OnlineSecurity",
    "OnlineBackup",
    "DeviceProtection",
    "TechSupport",
    "StreamingTV",
    "StreamingMovies",
    "Contract",
    "PaperlessBilling",
]

stringIndexer = StringIndexer(inputCols=categoricalCols, outputCols=[x + "Index" for x in categoricalCols])
encoder = OneHotEncoder(inputCols=stringIndexer.getOutputCols(), outputCols=[x + "OHE" for x in categoricalCols])

from pyspark.ml.feature import QuantileDiscretizer

# Apply QuantileDiscretizer to the TotalCharges column
discretizer = QuantileDiscretizer(numBuckets=4, inputCol="TotalCharges", outputCol="TotalChargesBucket")

# Fit the discretizer and transform the data
spark_dr = discretizer.fit(spark_dr).transform(spark_dr)

# Show the transformed data
spark_dr.select("TotalCharges", "TotalChargesBucket").display()

labelToIndex = StringIndexer(inputCol="Churn", outputCol="label")

stringIndexerModel = stringIndexer.fit(trainDF)
display(stringIndexerModel.transform(trainDF))

In [ ]:
# Task 2.4	Manipulate the feature engineering, using the transformers Imputer, StringIndexer, QuantileDiscretizer. Use VectorAssembler to give input to the model, you may take note, to use suitable numerical columns to end the feature engineering stage.


In [ ]:
# Task 2.5	Combine the pipeline fit, configure the train, test data and apply suitable regression model for the label and features columns. Execute the model and print the results of accuracy, plot the output with Area Under ROC. Evaluate the model with Binaryclassifiermetrics to analyze the prediction.